In [1]:
import numpy as np 
import pandas as pd 
import os
import spacy
import string
import re
import numpy as np
from spacy.symbols import ORTH
from collections import Counter
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import numpy as np
from collections import defaultdict
import re
from sklearn.metrics import confusion_matrix

## Read file into DataFrame

In [2]:
# read the file into DataFrame
df = pd.read_csv('./CAMEO_IDEA_labeled_data.csv')

# separate content and label
text = df['Content']
labels = df['Category Code']

## Tokenize the text

In [3]:
# funtion tokenize sentence
tokenizer = spacy.load("en_core_web_sm")
stopwords = tokenizer.Defaults.stop_words
# tokenize, lemmatize the text, drop punctuations and stopwords
tokenize = lambda t: [token.lemma_ for token in tokenizer(t) if (not token.is_punct) and (not token.is_stop)]

# only tokenize the text
# tokenize = lambda t: [token.text for token in tokenizer(t)]

In [4]:
# build dictionary <key=word : value=count>
cnt = Counter()
size = text.size
for idx in range(size):
    for word in tokenize(text[idx]):
        cnt[word] += 1

In [5]:
counter = dict(cnt)

### Optional

In [5]:
# filter out low-frequency word
min_threshold = 1
count = {x: count for x, count in cnt.items() if count >= min_threshold}

In [6]:
# filter out high-frequency word
max_threshold = 1
count = {x: count for x, count in cnt.items() if count <= max_threshold}

## Split dataset into train set and test set

In [6]:
X = np.array(text)
y = np.array(labels)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

## Prepare for word embedding

In [8]:
# download glove dictionary
# def download_glove():
#     ! wget http://nlp.stanford.edu/data/glove.6B.zip
#     ! unzip glove.6B.zip -C data
    
# download_glove()
# ! unzip glove.6B.zip

In [9]:
# load word embedding dictionary (<key=word : value=vector>)
def load_embedding_dict():
    embeddings_dict = {}
    with open("glove.6B.50d.txt", 'r') as file:
        for line in file:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], "float32")
            embeddings_dict[word] = vector
    return embeddings_dict

glove_dic = load_embedding_dict()

In [10]:
# create dictionaries(<key=word : value=index number>) (<key=word : value=vector>)
def create_embedding_matrix(count,emb_size=50):
    size = len(count) + 2
    word_idx_dict = {}
    word_vec = np.zeros((size, emb_size), dtype="float32")
    
    # add padding and UNK keyword
    word_idx_dict[""] = 0
    word_vec[0] = np.zeros(emb_size, dtype='float32')
    word_idx_dict["UNK"] = 1
    word_vec[1] = np.random.uniform(-0.25, 0.25, emb_size)

    for i, word in enumerate(count.keys()):
        word_idx_dict[word] = i + 2

        if word in glove_dic:
            word_vec[i + 2] = glove_dic[word]
        else:
            word_vec[i + 2] = np.random.uniform(-0.25,0.25, emb_size)

    return word_idx_dict, word_vec
    
word_idx_dict, pretrained_weight = create_embedding_matrix(count)

## Prepare for encoding sentence

In [11]:
def encode_sentence(line, word_idx_dict, N=400, padding_start=True):
    tokens = tokenize(line)
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([word_idx_dict.get(word, word_idx_dict["UNK"]) for word in tokens])
    length = min(N, len(enc1))
    if padding_start:
        enc[:length] = enc1[:length]
    else:
        enc[N - length:] = enc1[:length]
    return enc, length

## Build DataSet and DataLoader for model

In [12]:
class EventDataset(Dataset):
    def __init__(self, X, y, N=40, padding_start=False):
        self.y = y
        self.X = [encode_sentence(line, word_idx_dict, N, padding_start) for line in X]
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x, s = self.X[idx]
        return x, s, self.y[idx]

In [13]:
train_ds = EventDataset(X_train, y_train)
valid_ds = EventDataset(X_val, y_val)
train_dl = DataLoader(train_ds, batch_size=30, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=30)

## Training loop

In [52]:
def update_optimizer(optimizer, lr):
    for i, param_group in enumerate(optimizer.param_groups):
        param_group["lr"] = lr

In [186]:

def train_epocs(model, optimizer, train_dl, val_dl, epochs=10):
    global max_acc
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            # s is not used in this model
            x = x.long() 
            y = y.float() 
#             x = x.long()
#             y = y.float()
            y_pred = model(x)
            
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics(model, val_dl)
        if val_acc > max_acc:
            max_acc = val_acc
        if i % 5 == 0:
            print("train loss %.3f val loss %.3f and val accuracy %.3f" % (sum_loss/total, val_loss, val_acc))
#             print(y.numpy())
            
#             print()
#             print(confusion_matrix(y.numpy(), F.softmax(y_pred).detach().numpy()))

## Evaluation

In [187]:
def val_metrics(model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, s, y in valid_dl:
        # s is not used here
        x = x.long()
        y = y.float().unsqueeze(1)
#         x = x.long()
#         y = y.float().unsqueeze(1)
        y_hat = model(x)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        y_pred = y_hat > 0
        correct += (y_pred.float() == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

## Define the model

In [188]:
class GRUModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, glove_weights=None) :
        super(GRUModel,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        if glove_weights is not None:
            self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
            self.embeddings.weight.requires_grad = True ## freeze embeddings
            
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, 1)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
#         print(x.size())
        out_pack, ht = self.gru(x)
        x = self.linear(ht[-1])
#         print(x.size())
        return x

In [190]:
for i in range(4): 
    vocab_size = len(word_idx_dict)
    model = GRUModel(vocab_size, 50, 50*(i+1), glove_weights = pretrained_weight)
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=0.01)
    max_acc = 0
    train_epocs(model, optimizer, train_dl, valid_dl, epochs=30)
    print(max_acc)
    
    vocab_size = len(word_idx_dict)
    model = GRUModel(vocab_size, 50, 50*(i+1), glove_weights = pretrained_weight)
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=0.1)
    max_acc = 0
    train_epocs(model, optimizer, train_dl, valid_dl, epochs=30)
    print(max_acc)

train loss 0.357 val loss 0.298 and val accuracy 0.899
train loss 0.069 val loss 0.275 and val accuracy 0.899
train loss 0.046 val loss 0.362 and val accuracy 0.892
train loss 0.039 val loss 0.358 and val accuracy 0.886
train loss 0.032 val loss 0.367 and val accuracy 0.899
train loss 0.037 val loss 0.420 and val accuracy 0.892
tensor(0.9051)
train loss 0.376 val loss 0.246 and val accuracy 0.911
train loss 0.082 val loss 0.562 and val accuracy 0.892
train loss 0.054 val loss 0.444 and val accuracy 0.892
train loss 0.091 val loss 0.560 and val accuracy 0.905
train loss 0.076 val loss 0.340 and val accuracy 0.886
train loss 0.074 val loss 0.607 and val accuracy 0.899
tensor(0.9114)
train loss 0.319 val loss 0.273 and val accuracy 0.911
train loss 0.086 val loss 0.277 and val accuracy 0.899
train loss 0.068 val loss 0.316 and val accuracy 0.899
train loss 0.040 val loss 0.318 and val accuracy 0.918
train loss 0.037 val loss 0.324 and val accuracy 0.905
train loss 0.030 val loss 0.347 and

In [191]:
class GRUModel1(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, hidden_dim1, glove_weights=None) :
        super(GRUModel1,self).__init__()
        self.hidden_dim = hidden_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        if glove_weights is not None:
            self.embeddings.weight.data.copy_(torch.from_numpy(glove_weights))
            self.embeddings.weight.requires_grad = True ## freeze embeddings
            
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim1)
        self.linear1 = nn.Linear(hidden_dim1, 1)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = self.embeddings(x)
        x = self.dropout(x)
#         print(x.size())
        out_pack, ht = self.gru(x)
        x = self.linear(ht[-1])
        x = self.linear1(x)
        return x

In [192]:
for i in range(4):
    model1 = GRUModel1(vocab_size, 50, 50*(i+1), 50, glove_weights = pretrained_weight)
    parameters1 = filter(lambda p: p.requires_grad, model1.parameters())
    optimizer1 = torch.optim.Adam(parameters1, lr=0.01)
    max_acc = 0
    train_epocs(model1, optimizer1, train_dl, valid_dl, epochs=30)
    print(max_acc)
    
    model1 = GRUModel1(vocab_size, 50, 50*(i+1), 50, glove_weights = pretrained_weight)
    parameters1 = filter(lambda p: p.requires_grad, model1.parameters())
    optimizer1 = torch.optim.Adam(parameters1, lr=0.1)
    max_acc = 0
    train_epocs(model1, optimizer1, train_dl, valid_dl, epochs=30)
    print(max_acc)

train loss 0.351 val loss 0.273 and val accuracy 0.905
train loss 0.068 val loss 0.472 and val accuracy 0.918
train loss 0.042 val loss 0.281 and val accuracy 0.924
train loss 0.033 val loss 0.331 and val accuracy 0.918
train loss 0.030 val loss 0.416 and val accuracy 0.918
train loss 0.039 val loss 0.419 and val accuracy 0.880
tensor(0.9304)
train loss 1.662 val loss 0.459 and val accuracy 0.899
train loss 0.240 val loss 0.352 and val accuracy 0.892
train loss 0.076 val loss 0.434 and val accuracy 0.886
train loss 0.074 val loss 1.062 and val accuracy 0.481
train loss 0.087 val loss 0.434 and val accuracy 0.873
train loss 0.036 val loss 0.586 and val accuracy 0.886
tensor(0.9114)
train loss 0.430 val loss 0.282 and val accuracy 0.905
train loss 0.090 val loss 0.267 and val accuracy 0.918
train loss 0.048 val loss 0.247 and val accuracy 0.924
train loss 0.030 val loss 0.357 and val accuracy 0.924
train loss 0.032 val loss 0.368 and val accuracy 0.924
train loss 0.032 val loss 0.366 and